## [Repast for Python (Repast4Py) User Guide](https://repast.github.io/repast4py.site/guide/user_guide.html)

## [API](https://repast.github.io/repast4py.site/apidoc/index.html)

## [GitHub Repast/repast4py](https://github.com/Repast/repast4py)

## [MPI for Python](https://mpi4py.readthedocs.io/en/stable/tutorial.html#collective-communication)

# An idea for the initial example: 

### something close to the [Chakraborti model](https://www.scientificamerican.com/article/is-inequality-inevitable/) of Winners, Losers 

ESC L set or unset row numbers

## 0

program starter

In [1]:
%%writefile starter.py

from repast4py import parameters
from runMod import *

# https://repast.github.io/repast4py.site/apidoc/source/repast4py.parameters.html
"""
create_args_parser()
Creates an argparse parser with two arguments: 
1) a yaml format file containing model parameter input, and 
2) an optional json dictionary string that can override that input.
"""
parser = parameters.create_args_parser()

args = parser.parse_args()
#print(1,args,flush=True)

"""
init_params(parameters_file, parameters)
Initializes the repast4py.parameters.params dictionary with the model input parameters.
"""
params = parameters.init_params(args.parameters_file, args.parameters)
#print(2,params,flush=True)

run(params)

Overwriting starter.py


## 1 

run the model

In [2]:
%%writefile runMod.py

from typing import Dict
from model import *

def run(params: Dict):
    #print(3, params)
    #print (4, params['random.seed'])
    #print (4.1, params['a'])
    
    model = Model(MPI.COMM_WORLD, params)
    model.start()

Overwriting runMod.py


## 2

agent definition

In [3]:
%%writefile WinnerLoser.py

from repast4py import core

class WinnerLoser(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, wallet: float):
        super().__init__(id=local_id, type=WinnerLoser.TYPE, rank=rank)
        self.myWallet = wallet


Overwriting WinnerLoser.py


## 3

the model

In [4]:
%%writefile model.py

from mpi4py import MPI
from typing import Dict
from repast4py import schedule
from repast4py import context as ctx
import repast4py
import time
"""
time() -> floating point number
        Return the current time in seconds since the Epoch.
        Fractions of a second may be present if the system clock provides them.
to know the Epoch, time.gmtime(0) (in Unix: 19700101)
"""
from WinnerLoser import *

class Model:
    """
    The Model class encapsulates the simulation, and is
    responsible for initialization (scheduling events, creating agents,
    and the grid the agents inhabit IF ANY), and the overall iterating
    behavior of the model.

    Args:
        comm: the mpi communicator over which the model is distributed.
        params: the simulation input parameters
    """
    def __init__(self, comm: MPI.Intracomm, params: Dict):

        self.rank    = comm.Get_rank()
        self.rankNum = comm.Get_size() #pt
        
        #print(5, "rank", self.rank, "rank number", self.rankNum)
        
        
        # create the context to hold the agents and manage cross process
        # synchronization
        self.context = ctx.SharedContext(comm)
        
        # create the schedule
        # https://repast.github.io/repast4py.site/apidoc/source/repast4py.schedule.html
        
        self.countStep=-1
        
        """
        init_schedule_runner(comm)
        Initializes the default schedule runner, a dynamic schedule of executable 
        events shared and synchronized across processes.
        Events are added to the scheduled for execution at a particular tick. 
        The first valid tick is 0. Events will be executed in tick order, earliest 
        before latest. Events scheduled for the same tick will be executed in the 
        order in which they were added. If during the execution of a tick, 
        an event is scheduled before the executing tick (i.e., scheduled to occur in 
        the past) then that event is ignored. The scheduled is synchronized across 
        process ranks by determining the global cross-process minimum next scheduled 
        event time, and executing only the events schedule for that time. In this way, 
        no schedule runs ahead of any other.
        """
        self.runner = schedule.init_schedule_runner(comm)
        
        """
        schedule_repeating_event(at, interval, evt)
        Schedules the specified event to execute at the specified tick, and repeat at 
        the specified interval.

        Parameters
        at (float) – the time of the event.
        interval (float) – the interval at which to repeat event execution.
        evt (Callable) – the Callable to execute when the event occurs.

            A callable is anything that can be called.
            The built-in callable (PyCallable_Check in objects.c) checks if the argument 
            is either:
                an instance of a class with a __call__ method or
                is of a type that has a non null tp_call (c struct) member which 
                indicates callability otherwise (such as in functions, methods etc.)
        """
        self.runner.schedule_repeating_event(0, 1, self.step)
        
        """
        schedule_stop(at)
        Schedules the execution of this schedule to stop at the specified tick.

        Parameters
        at (float) – the tick at which the schedule will stop.
        """
        self.runner.schedule_stop(params['stop.at'])
        
        self.runner.schedule_end_event(self.finish)
        

        
        # create agents
        
        # https://repast.github.io/repast4py.site/apidoc/source/repast4py.random.html
        """
        Random numbers for repast4py. When this module is imported, repast4py.random.default_rng 
        is created using the current epoch time as the random seed, and repast4py.random.seed is 
        set to that value. 
        
        repast4py.random.init(rng_seed=None)
        Initializes the default random number generator using the specified seed.
        """
        rng = repast4py.random.default_rng  #da capire & seed qui?
        
        
        for i in range(params['WinnerLoser.count'] // self.rankNum): #to subdivide the total #pt
            # create and add the agent to the context
            aWallet=100 * rng.random()
            #print(aWallet,flush=True)
            winnerLoser = WinnerLoser(i,self.rank,aWallet)
            self.context.add(winnerLoser)
            
        #print(6, "agents in rank",self.rank, "=", len(self.context.agents(0, \
        #                                              shuffle=True)),flush=True)
        
    def step(self):
        
        """
        synchronize(restore_agent, sync_ghosts=True)
        Synchronizes the model state across processes by moving agents, 
        filling projection buffers with ghosts, updating ghosted state and so forth.

        Parameters
        restore_agent (Callable) – a callable that takes agent state data and 
        returns an agent instance from that data. The data is a tuple whose first 
        element is the agent’s unique id tuple, and the second element is the 
        agent’s state, as returned by that agent’s type’s save() method.

        sync_ghosts (bool) – if True, the ghosts in any SharedProjections and 
        value layers associated with this SharedContext are also synchronized. 
        Defaults to True.
        """
        self.context.synchronize(self.fake)  # assures more regular steps of
                                             # cyclea among ranks??
        
        self.countStep+=1
        
        tick = self.runner.schedule.tick
        
        print("rank",self.rank,"step",self.countStep,"in tick",tick,\
              "clock",time.time(),flush=True)
        
    def fake(self):
        pass
    
    def finish(self):
        tick = self.runner.schedule.tick
        print("ciao by rank",self.rank,"step",self.countStep,"in tick",tick,\
              "clock",time.time(),flush=True)
        
    def start(self):
        self.runner.execute()
        

Overwriting model.py


# Execution

In [5]:
# R U N
# %sx mpirun -n 1 python3 starter.py winners-losers.yaml
! mpirun -n 3 python3 starter.py winners-losers.yaml
# %sx mpirun -n 1 python3 starter.py winners-losers.yaml '{"a": 123,"random.seed": 1000}'

rank 0 step 0 in tick 0 clock 1663681671.7793012
rank 1 step 0 in tick 0 clock 1663681671.7793138
rank 2 step 0 in tick 0 clock 1663681671.779318
rank 0 step 1 in tick 1 clock 1663681671.7796118
rank 2 step 1 in tick 1 clock 1663681671.779636
rank 1 step 1 in tick 1 clock 1663681671.779634
rank 0 step 2 in tick 2 clock 1663681671.779809
rank 2 step 2 in tick 2 clock 1663681671.779823
rank 1 step 2 in tick 2 clock 1663681671.77983
rank 0 step 3 in tick 3 clock 1663681671.7800808
rank 1 step 3 in tick 3 clock 1663681671.78009
rank 2 step 3 in tick 3 clock 1663681671.780096
rank 0 step 4 in tick 4 clock 1663681671.7802398
rank 2 step 4 in tick 4 clock 1663681671.780254
rank 1 step 4 in tick 4 clock 1663681671.78026
rank 0 step 5 in tick 5 clock 1663681671.780433
rank 1 step 5 in tick 5 clock 1663681671.780451
rank 2 step 5 in tick 5 clock 1663681671.780454
rank 0 step 6 in tick 6 clock 1663681671.780591
rank 1 step 6 in tick 6 clock 1663681671.780606
rank 2 step 6 in tick 6 clock 16636816